# Accessing Multisprectral Satellite Imagery from Copernicus Data Space Ecosystem with Sentinel Hub API

In [1]:
### poss necessary: %pip install sentinelhub[AWS]


The Sentinel Hub API is a RESTful API interface that provides access to various satellite imagery archives. It allows you to access raw satellite data, rendered images, statistical analysis, and other features. 

To use the features in this notebook you need to visit https://dataspace.copernicus.eu and create an account with Copernicus, the official governing body of Sentinel Missions for the European Space Agency (ESA).

In [2]:
import os
from sentinelhub import (SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)
from dotenv import load_dotenv
import requests_oauthlib as requests
import matplotlib.pyplot as plt
import numpy as np
import datetime 
import pandas as pd
import tqdm

/Users/sara_mac/Desktop/projects/plastic_detection/Sentinel2PlasticDetectionProject/omdena/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Credentials

Credentials for Sentinel Hub services (`client_id` & `client_secret`) can be obtained in your [Dashboard](https://shapps.dataspace.copernicus.eu/dashboard/#/). In the User Settings you can create a new OAuth Client to generate these credentials. For more detailed instructions, visit the relevant [documentation page](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html).

Now that you have your `client_id` & `client_secret`,  you can proceed from here. 

In [3]:
SHConfig.get_config_location()

'/Users/sara_mac/.config/sentinelhub/config.toml'

In [9]:
# Display the existing configuration
config = SHConfig()
config

TOMLDecodeError: Expected newline or end of document after a statement (at line 3, column 53)

In [ ]:
# You need to replace the client ID and secret with your own credentials in your .env file
# in the same directory as this script, by simply inserting the following lines:
# CLIENT_ID= "your_client_id"
# CLIENT_SECRET= "your_client_secret"
# TOKEN_URL= "https://your_token_url"
# BASE_URL= "https://your_base_url"

load_dotenv() 
config = SHConfig()
config.sh_client_id = os.getenv("CLIENT_ID")
config.sh_client_secret = os.getenv("CLIENT_SECRET")
config.sh_token_url = os.getenv("TOKEN_URL")
config.sh_base_url = os.getenv("BASE_URL")


In [ ]:
# check that the credentials are set correctly 
if not config.sh_client_id or not config.sh_client_secret:
    print("Please provide your Sentinel Hub credentials in the .env file.")
    exit(1)

# check that the credentials are what you expect (i.e. output is the same & not None)
# NOTE: you can also set the credentials directly in the code, 
# but this is not recommended for security reasons.
# do not print the secret credentials 'id' or 'secret' in a public notebook for security reasons either
print(os.getenv("TOKEN_URL"))   
print(os.getenv("BASE_URL"))

Instructions on how to configure your Sentinel Hub Python package can be found [here](https://sentinelhub-py.readthedocs.io/en/latest/configure.html). Using these instructions you can create a profile specific to using the package for accessing Copernicus Data Space Ecosystem data collections. This is useful as changes to the the config class are usually only temporary in your notebook and by saving the configuration to your profile you won't need to generate new credentials or overwrite/change the default profile each time you rerun or write a new Jupyter Notebook. 

In [ ]:
#Po River 
AOI = 'Po River Plume'
aoi_coords_wgs84 = [12.24, 44.66, 12.0, 45.077] 

In [ ]:
resolution = 10
aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)
print(f"Image shape at {resolution} m resolution: {aoi_size} pixels")

Image shape at 10 m resolution: (2060, 4565) pixels


In [ ]:
catalog = SentinelHubCatalog(config=config)

In [ ]:
# Retrieve images from the Sentinel Hub Catalog for the specified area of interest
#  and time interval of July 2019

aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
time_interval = "2019-07-01", "2019-07-31"

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L1C,
    bbox=aoi_bbox,
    time=time_interval,
    fields={"include": ["id", "properties.datetime"], "exclude": []},
)

results = list(search_iterator)
print("Total number of results:", len(results))


Total number of results: 24


[{'id': 'S2A_MSIL1C_20190730T100031_N0500_R122_T32TQQ_20230710T202111.SAFE',
  'properties': {'datetime': '2019-07-30T10:08:32.85Z'}},
 {'id': 'S2A_MSIL1C_20190730T100031_N0500_R122_T32TQR_20230710T202111.SAFE',
  'properties': {'datetime': '2019-07-30T10:08:17.911Z'}},
 {'id': 'S2B_MSIL1C_20190728T101029_N0500_R022_T32TQQ_20230710T194856.SAFE',
  'properties': {'datetime': '2019-07-28T10:18:33.245Z'}},
 {'id': 'S2B_MSIL1C_20190728T101029_N0500_R022_T32TQR_20230710T194856.SAFE',
  'properties': {'datetime': '2019-07-28T10:18:18.692Z'}},
 {'id': 'S2B_MSIL1C_20190725T100039_N0500_R122_T32TQQ_20230619T024542.SAFE',
  'properties': {'datetime': '2019-07-25T10:08:36.543Z'}},
 {'id': 'S2B_MSIL1C_20190725T100039_N0500_R122_T32TQR_20230619T024542.SAFE',
  'properties': {'datetime': '2019-07-25T10:08:21.601Z'}},
 {'id': 'S2A_MSIL1C_20190723T101031_N0500_R022_T32TQQ_20230718T015529.SAFE',
  'properties': {'datetime': '2019-07-23T10:18:30.071Z'}},
 {'id': 'S2A_MSIL1C_20190723T101031_N0500_R022_T3

Check if any of the id's match products listed in LM_centroids.xlxs

In [ ]:
# Function to match LW source data to the image ID 
# by removing S2A_MSIL1C_ from the 'id' and keeping only the datestr and codeT which are
# the same as in the LM_centroids.xlsx file i.e. naming convention: 
# S2A_MSIL1C__YYYYMMDDTXXXXXX_....


def check_matching_ids(results, lm_centroids_path):
    # Load the LM_centroids.xlsx file
    lm_centroids = pd.read_excel(lm_centroids_path)
    
    # Ensure the column 'Str_time' exists in the Excel file
    if 'Str_time' not in lm_centroids.columns:
        raise ValueError("The column 'Str_time' is not found in the provided Excel file.")
    
    # Extract the 'Str_time' column as a set for faster lookup
    str_time_set = set(lm_centroids['Str_time'])
    
    # Iterate through the results and check for matches
    matching_ids = []
    for result in results:
        trimmed_id = result['id'][11:26]
        if trimmed_id in str_time_set:
            matching_ids.append(result['id'])
    
    return matching_ids


lm_centroids_path = "../LM_centroids.xlsx"
matching_ids = check_matching_ids(results, lm_centroids_path)
print("Matching IDs:", matching_ids)

Matching IDs: ['S2A_MSIL1C_20190730T100031_N0500_R122_T32TQQ_20230710T202111.SAFE', 'S2A_MSIL1C_20190730T100031_N0500_R122_T32TQR_20230710T202111.SAFE', 'S2B_MSIL1C_20190725T100039_N0500_R122_T32TQQ_20230619T024542.SAFE', 'S2B_MSIL1C_20190725T100039_N0500_R122_T32TQR_20230619T024542.SAFE', 'S2A_MSIL1C_20190723T101031_N0500_R022_T32TQQ_20230718T015529.SAFE', 'S2A_MSIL1C_20190723T101031_N0500_R022_T32TQR_20230718T015529.SAFE', 'S2A_MSIL1C_20190720T100031_N0500_R122_T32TQQ_20230715T235221.SAFE', 'S2A_MSIL1C_20190720T100031_N0500_R122_T32TQR_20230715T235221.SAFE', 'S2B_MSIL1C_20190705T100039_N0500_R122_T32TQQ_20230718T022116.SAFE', 'S2B_MSIL1C_20190705T100039_N0500_R122_T32TQR_20230718T022116.SAFE', 'S2A_MSIL1C_20190703T101031_N0500_R022_T32TQQ_20230717T022229.SAFE', 'S2A_MSIL1C_20190703T101031_N0500_R022_T32TQR_20230717T022229.SAFE']


In [ ]:
print(f"{len(matching_ids)} matching images found between {time_interval[0]} and {time_interval[1]} for {AOI}.")

12 matching images found between 2019-07-01 and 2019-07-31 for Po River Plume.


In [ ]:

evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 11
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

request_true_color = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                name="s2l1c", service_url="https://sh.dataspace.copernicus.eu"
            ),
            time_interval=("2019-07-01", "2019-07-20"),
            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
    bbox=aoi_bbox,
    size=aoi_size,
    config=config,
)

based on Booth et al. 4 bands were selected:
"The model trained with only 4 bands (the ones contributing to calculating FDI and NDVI) demonstrated good performance than using all 13 Sentinel-2 bands. However, it is possible that other band combinations could improve model performance. Removing some of the lower resolution bands, as well as bands where wavelengths do not correlate with plastic materials, may reduce noise in the data set."

In [ ]:
evalscript_true_color = """
//VERSION=3

function setup() {
    return {
        input: [{
            bands: ["B04", "B06", "B08", "B11"] 
        }],
        output: {
            bands: 4
        }
    };
}

function evaluatePixel(sample) {
    return [sample.B04, sample.B06, sample.B08, sample.B11]; // RGB + NIR + SWIR
}
"""

# Generate requests for each matching ID
requests_true_color = []
for matching_id in matching_ids:
    request = SentinelHubRequest(
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                identifier=matching_id,
            time_interval=time_interval,
            other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=aoi_bbox,
        size=aoi_size,
        config=config,
    )
    requests_true_color.append(request)

print(f"Generated {len(requests_true_color)} requests for true color images.")

Generated 12 requests for true color images.


In [ ]:
true_color_imgs = requests_true_color[0].get_data()
print(
    f"Returned data is of type = {type(true_color_imgs)} and length {len(true_color_imgs)}."
)
print(
    f"Single element in the list is of type {type(true_color_imgs[-1])} and has shape {true_color_imgs[-1].shape}"
)

DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
401 Client Error: Unauthorized for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"status": 401, "reason": "Unauthorized", "message": "You are not authorized! Please provide a valid access token within the header [Authorization: Bearer <accessToken>] of your request.", "code": "COMMON_UNAUTHORIZED"}"

In [ ]:
image = true_color_imgs[0]
print(f"Image type: {image.dtype}")

# plot function
# factor 1/255 to scale between 0-1
# factor 3.5 to increase brightness
plot_image(image, factor=3.5 / 255, clip_range=(0, 1))

Other Areas next (and other time periods tbd):

In [ ]:
AOI = 'North Corsica'
aoi_coords_wgs84 = [9.288940,43.012681,9.481888,43.119530] 

In [ ]:
Optional AOI #3:
South East Calabria
(16 38.75, 16 37.75, 17 37.75, 17 38.75, 16 38.75)